In [1]:
import os
import sys
BASE = '/home/wb/smp2018'
sys.path.append(BASE)

In [2]:
import json
import pandas as pd
from init.config import Config
from pyltp import SentenceSplitter
from pyltp import Segmentor
from sklearn.model_selection import train_test_split

In [3]:
LTP_DATA_DIR = '/home/wb/ltp_data_v3.4.0'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model

In [4]:
data_base = '/home/wb/text-classification/data'

src = data_base + '/News_label.txt'

all_path = data_base + '/all.csv'

train_path = data_base + '/train.csv'

test_path = data_base + '/test.csv'

In [9]:
def create_training_data(save_all_samples=False, save_split_sample=False):
    lines = []
    with open(src) as f:
        lines = f.readlines()
    
    segmentor = Segmentor()  # 初始化实例
    segmentor.load(cws_model_path)  # 加载模型

    data = []
    total = len(lines)
    one = total // 100
    for i, l in enumerate(lines):
        if len(l) < 3: continue
        label, content = l.split('\t')
        label = int(label)
        content = content.strip().replace('\t', ' ')
        
        content = ' '.join(segmentor.segment(content))
        
        js = {}
        js['label'] = label
        js['content'] = content
        
        data.append(js)
        
        if i % one == 0:
            print('cut processing %s %%' % str((i * 100) // total))
    
    segmentor.release()  # 释放模型

    data = pd.DataFrame(data)

    print("total samples number:", len(data))
    
    if save_all_samples:
        data.fillna("", inplace=True)
        data.to_csv(all_path, index=False, sep='\t')
        
    train, val = train_test_split(data, test_size=0.1, shuffle=True, random_state=1)
    print("train samples number:", len(train))
    print("vali samples number:", len(val))
    if save_split_sample:
        train.to_csv(train_path, index=False, sep='\t')
        val.to_csv(test_path, index=False, sep='\t')
    return train, val

In [25]:
def get_train_split_data():
    train = pd.read_csv(train_path, sep='\t')
    test = pd.read_csv(test_path, sep='\t')
    return train, test

- total samples number: 48,480
- train samples number: 43,632
- vali samples number: 4,848

In [19]:
# train, val = create_training_data(True, True)

In [20]:
def get_train_all_data():
    train_all_data = pd.read_csv(all_path, sep='\t')
    return train_all_data

In [23]:
# all_df = get_train_all_data()

In [24]:
# all_df

In [28]:
# train, test = get_train_split_data()

In [29]:
# train